In [ ]:
# import tools
import pandas as pd
%matplotlib inline
import qiime2
from qiime2 import Artifact
from tempfile import mkdtemp
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition
from qiime2.plugins import fragment_insertion
from qiime2.plugins.fragment_insertion.methods import filter_features
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.visualizers import summarize
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pylab
from scipy.optimize import curve_fit

## Import data

In [ ]:
# import raw feature table and taxonomy
table = Artifact.load('96057_feature-table.qza')
taxonomy = Artifact.load('96057_reference-hit.taxonomy_gg.qza')

## Collapse and subset data

In [ ]:
# collapsed table to genus level
t6 = taxa.methods.collapse(table = table,
                           taxonomy = taxonomy,
                           level = 6)
# import collapsed table as pandas dataframe
df = t6.collapsed_table.view(pd.DataFrame)
# subset out Zymo mock community samples
zymo = df[df.index.str.contains('Zymo')]
# ensure table values are numeric
zymo = zymo.astype(float)

In [ ]:
# quick visual check that top 8 taxa make up most of the reads in highest input sample (well A7)
max_input = zymo[zymo.index.str.contains("A7")]
zymoT = max_input.T
zymoT.sort_values(zymoT.columns[0], ascending = False).head(10)

## Caluclate reads aligning to mock community

In [ ]:
# make a list of the taxa expected in the zymo community (also the  most abundant in the highest input samples)
zymo7taxa = zymoT.sort_values(zymoT.columns[0], ascending = False).head(7).index

In [ ]:
# Calculate the total number of reads per sample
zymo['deblur_reads'] = zymo.sum(axis=1)
# calculate the number of reads aligning to the mock community input genera
zymo['zymo_reads'] = zymo[zymo7taxa].sum(axis=1)
# calculate the percent correctly assigned
zymo['correct_assign'] = zymo['zymo_reads'] / zymo['deblur_reads']

In [ ]:
# subset out KatharoSeq columns and add in logarithim of reads for plotting
katharo = zymo[['correct_assign','deblur_reads','zymo_reads']]
katharo['log_deblur_reads'] = np.log10(katharo['deblur_reads'])

## Fit an allosteric sigmoid curve for extrpolating min read count #

In [ ]:
# define the allosteric sigmoid equation
def allosteric_sigmoid(x, h, k_prime):
    y = x ** h / (k_prime + x ** h)
    return y
 
# fit the curve to your data
popt, pcov = curve_fit(allosteric_sigmoid, katharo['log_deblur_reads'], katharo['correct_assign'], method='dogbox')
print(popt)
# plot fit curve
x = np.linspace(0, 5, 50)
y = allosteric_sigmoid(x, *popt)

# plot the fit
pylab.plot(katharo['log_deblur_reads'], katharo['correct_assign'], 'o', label='data')
pylab.plot(x,y, label='fit')
pylab.ylim(0, 1.05)
pylab.ylabel('%reads aligning to mock community')
pylab.xlabel('log(quality-filtered reads)')
pylab.legend(loc='best')
pylab.show()

In [ ]:
# Determine the number of reads at which 50% of reads are expected to match input

# assign variables and solve for X (number of reads to pass filter)
h = popt[0]  # first value printed above graph
k = popt[1]   # second value printed above graph
y = 0.5 ## what you want to solve for

min_log_reads = np.power((k/(1/y-1)),(1/h))
min_freq_50 = np.power(10, min_log_reads).astype(int)
min_freq_50

In [ ]:
# Determine the number of reads at which 80% of reads are expected to match input

# assign variables and solve for X (number of reads to pass filter)
h = popt[0]  # first value printed above graph
k = popt[1]   # second value printed above graph
y = 0.8 ## what you want to solve for

min_log_reads = np.power((k/(1/y-1)),(1/h))
min_freq_80 = np.power(10, min_log_reads).astype(int)
min_freq_80

In [ ]:
# Determine the number of reads at which 90% of reads are expected to match input

# assign variables and solve for X (number of reads to pass filter)
h = popt[0]  # first value printed above graph
k = popt[1]   # second value printed above graph
y = 0.9 ## what you want to solve for

min_log_reads = np.power((k/(1/y-1)),(1/h))
min_freq_90 = np.power(10, min_log_reads).astype(int)
min_freq_90

## Remove samples with less than Katharoseq read limit

In [ ]:
# filter out samples with read counts below what is estimated to achieve 50% accuracy  
KS_table_50 = feature_table.methods.filter_samples(table = table,
                             min_frequency = min_freq_50)
df_50 = KS_table_50.filtered_table.view(pd.DataFrame)

In [ ]:
# filter out samples with read counts below what is estimated to achieve 80% accuracy  
KS_table_80 = feature_table.methods.filter_samples(table = table,
                             min_frequency = min_freq_80)
df_80 = KS_table_80.filtered_table.view(pd.DataFrame)

In [ ]:
# filter out samples with read counts below what is estimated to achieve 90% accuracy  
KS_table_90 = feature_table.methods.filter_samples(table = table,
                             min_frequency = min_freq_90)
df_90 = KS_table_90.filtered_table.view(pd.DataFrame)

In [ ]:
# print the number (and percentage) of samples that are dropped at each filtering threshold
print("full dataset", len(df), "\n",
      "50%", len(df_50), len(df_50)/len(df),"%", "\n",
      "80%", len(df_80), len(df_80)/len(df),"%", "\n",
      "90%", len(df_90), len(df_90)/len(df),"%",)

### Export Data

In [ ]:
KS_table_80.filtered_table.save('96057_feature-table-KathSeqFil.qza')